<a href="https://colab.research.google.com/github/manor-s/DS-course-project-4/blob/main/Classification_Project_Manor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>